# Lab 01: Baseline Agent

## Overview

In this notebook, we deploy an **intentionally unoptimized** customer support agent to establish baseline metrics. This baseline will be used to measure improvements in subsequent notebooks.

**What you'll learn:**
- How to deploy an agent to AgentCore Runtime
- How to configure Langfuse for observability
- How to invoke agents via the AgentCore API
- How to establish baseline metrics for cost and latency

## Prerequisites

- AWS account with Bedrock and AgentCore access
- Langfuse account (free tier works)
- `.env` file configured with your credentials

## Workshop Journey

```
[01 Baseline] → 02 Quick Wins → 03 Caching → 04 Routing → 05 Guardrails → 06 Gateway → 07 Evaluations
     ↑
  You are here
```

## Step 1: Setup and Dependencies

Before starting, ensure you've run `uv sync` in the terminal and selected the `.venv` kernel in VS Code.

In [ ]:
from __future__ import annotations

import json
import os
import uuid
from pathlib import Path

from dotenv import load_dotenv

load_dotenv(override=True)

import boto3
from bedrock_agentcore_starter_toolkit import Runtime

region = os.environ.get("AWS_DEFAULT_REGION", "us-east-1")
control_client = boto3.client("bedrock-agentcore-control", region_name=region)
data_client = boto3.client("bedrock-agentcore", region_name=region)
agentcore_runtime = Runtime()

print(f"Region: {region}")
print(f"Langfuse Host: {os.environ.get('LANGFUSE_BASE_URL', 'Not set')}")

## Step 2: Review the Baseline Agent

Our baseline agent (`agents/v1_baseline.py`) is intentionally unoptimized:

- **Verbose system prompt** (~1500 tokens instead of ~250)
- **No max_tokens limit** (model can generate unlimited output)
- **No prompt caching** (system prompt processed every request)
- **No stop sequences** (model decides when to stop)

Let's examine the key parts of the baseline agent:

In [ ]:
# Read and display the baseline agent code
agent_file = Path("agents/v1_baseline.py")
print(agent_file.read_text())

## Step 3: Configure the Agent for Deployment

Now we'll configure the agent for deployment to AgentCore Runtime.

In [ ]:
agent_name = "customer_support_v1_baseline"
agent_file = str(Path("agents/v1_baseline.py").absolute())
requirements_file = str(Path("requirements-for-agentcore.txt").absolute())

print(f"Agent name: {agent_name}")
print(f"Agent file: {agent_file}")
print(f"Requirements: {requirements_file}")

In [ ]:
# Clean up any existing build files from previous labs
for f in ["Dockerfile", ".dockerignore", ".bedrock_agentcore.yaml"]:
    p = Path(f)
    if p.exists():
        p.unlink()
        print(f"Removed existing: {f}")

# Configure the runtime
print(f"Configuring agent: {agent_name}")
agentcore_runtime.configure(
    entrypoint=agent_file,
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file=requirements_file,
    region=region,
    agent_name=agent_name,
)

## Step 4: Modify Dockerfile for Langfuse

We need to disable the default OpenTelemetry instrumentation and use Langfuse instead.

In [ ]:
dockerfile_path = Path("Dockerfile")
if dockerfile_path.exists():
    content = dockerfile_path.read_text()
    # Replace opentelemetry-instrument wrapper with direct python call
    # Keep the correct module path: agents.v1_baseline
    if "opentelemetry-instrument" in content:
        import re

        content = re.sub(
            r'CMD \["opentelemetry-instrument", "python", "-m", "([^"]+)"\]', r'CMD ["python", "-m", "\1"]', content
        )
        dockerfile_path.write_text(content)
        print("Dockerfile modified for Langfuse")
    else:
        print("Dockerfile already configured or using different format")
else:
    print("Dockerfile not found - will be created during deployment")

## Step 5: Deploy to AgentCore Runtime

Deploy the agent with Langfuse environment variables.

In [ ]:
env_vars = {
    "LANGFUSE_BASE_URL": os.environ.get("LANGFUSE_BASE_URL"),
    "LANGFUSE_PUBLIC_KEY": os.environ.get("LANGFUSE_PUBLIC_KEY"),
    "LANGFUSE_SECRET_KEY": os.environ.get("LANGFUSE_SECRET_KEY"),
    "PYTHONUNBUFFERED": "1",
}

print("Deploying to AgentCore Runtime...")
print("This may take 5-10 minutes for the first deployment.")
launch_result = agentcore_runtime.launch(env_vars=env_vars, auto_update_on_conflict=True)
print(f"Agent deployed: {launch_result.agent_arn}")

In [ ]:
# Save the agent ARN for later use
agent_arn = launch_result.agent_arn
print(f"Agent ARN: {agent_arn}")

## Step 6: Test the Baseline Agent

Let's invoke the agent with some test queries to establish our baseline metrics.

In [ ]:
def invoke_agent(prompt):
    """Invoke the agent via AgentCore API."""
    response = data_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        runtimeSessionId=str(uuid.uuid4()),
        payload=json.dumps({"prompt": prompt}).encode(),
    )
    return json.loads(response["response"].read().decode("utf-8"))

In [ ]:
# Import Langfuse metrics helper
from utils.langfuse_metrics import (
    clear_metrics,
    collect_metric,
    get_latest_trace_metrics,
    print_metrics,
    print_metrics_table,
)

# Clear any previously collected metrics
clear_metrics()


def invoke_agent_with_metrics(prompt, test_name=""):
    """Invoke the agent and fetch + print Langfuse metrics."""
    response = invoke_agent(prompt)
    print(response)

    # Fetch and print metrics from Langfuse
    metrics = get_latest_trace_metrics(
        agent_name="customer-support-v1-baseline",
        wait_seconds=5,
        max_retries=5,
        timeout_seconds=120,
    )
    print_metrics(metrics, test_name)

    # Collect metrics for summary table
    collect_metric(metrics, test_name)

    return response, metrics

In [ ]:
# Standard test prompts - same across all notebooks for consistent comparison
TEST_PROMPTS = [
    # Single tool: get_return_policy
    ("Return Policy", "What is your return policy for laptops?"),
    # Single tool: get_product_info
    ("Product Info", "Tell me about your smartphone options"),
    # Single tool: get_technical_support (Bedrock KB)
    ("Technical Support", "My laptop won't turn on, can you help me troubleshoot?"),
    # Multi-tool: get_product_info + get_return_policy
    ("Multi-part Question", "I want to buy a laptop. What are the specs and what's the return policy?"),
    # No tool: General greeting
    ("General Question", "Hello! What can you help me with today?"),
]

# Run all tests and collect metrics
for test_name, prompt in TEST_PROMPTS:
    print("=" * 60)
    print(f"Test: {test_name}")
    print("=" * 60)
    response, metrics = invoke_agent_with_metrics(prompt, test_name=test_name)

In [ ]:
# Print summary table of all test metrics
print_metrics_table()

# Save metrics for comparison in later notebooks
from utils.langfuse_metrics import save_metrics
save_metrics("v1")

## Step 7: View Langfuse Dashboard

Now let's check the Langfuse dashboard to see our baseline metrics.

### What to look for:

1. **Token Usage**: Note the input and output token counts
2. **Latency**: Time taken for each request
3. **Cost**: Estimated cost per request
4. **No Cache Hits**: `cacheReadInputTokens` should be 0 (no caching)

### Dashboard URL

In [ ]:
langfuse_base_url = os.environ.get("LANGFUSE_BASE_URL", "https://cloud.langfuse.com")
print(f"View your traces at: {langfuse_base_url}")
print("\nFilter by tags: 'baseline', 'no-optimization'")
print("\nMetrics to record:")
print("- Average input tokens: _____")
print("- Average output tokens: _____")
print("- Average latency: _____ ms")
print("- Cache read tokens: 0 (expected)")

## Summary

In this notebook, we:

1. Deployed an unoptimized baseline agent to AgentCore Runtime
2. Configured Langfuse for observability
3. Ran test scenarios to establish baseline metrics
4. Identified areas for optimization:
   - Verbose system prompt (~1500 tokens)
   - No output token limits
   - No prompt caching
   - No model routing

**Next Steps:** In the next notebook, we'll apply "quick wins" optimizations:
- Concise system prompt
- max_tokens limit
- stop_sequences

**Next notebook:** [02-quick-wins.ipynb](./02-quick-wins.ipynb)

## Cleanup (Optional)

Uncomment and run if you want to delete the agent.

In [ ]:
# # Uncomment to delete resources created in this lab
# agentcore_runtime.destroy(delete_ecr_repo=True)
# print(f"Deleted agent and ECR repository: {agent_name}")